<a href="https://colab.research.google.com/github/emylincon/github/blob/main/github_contribution_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [2]:
x = np.array([5, 15, 25, 35, 45, 55]).reshape((-1, 1))
y = np.array([15, 11, 2, 8, 25, 32])


In [3]:
x

array([[ 5],
       [15],
       [25],
       [35],
       [45],
       [55]])

In [7]:
transformer = PolynomialFeatures(degree=2, include_bias=False)

In [8]:
x_ = transformer.fit_transform(x)


In [9]:
x_


array([[   5.,   25.],
       [  15.,  225.],
       [  25.,  625.],
       [  35., 1225.],
       [  45., 2025.],
       [  55., 3025.]])

In [10]:
model = LinearRegression().fit(x_, y)

In [11]:
r_sq = model.score(x_, y)
print(f"coefficient of determination: {r_sq}")


print(f"intercept: {model.intercept_}")


print(f"coefficients: {model.coef_}")

coefficient of determination: 0.8908516262498563
intercept: 21.37232142857144
coefficients: [-1.32357143  0.02839286]


In [45]:
# class creation
class Prep:
  def prepx(self, x:list, degree:int, include_bias:bool=False):
    if len(np.array(x).shape) == 1:
      x = np.array(x).reshape((-1, 1))
    transformer = PolynomialFeatures(degree=degree, include_bias=include_bias)
    return transformer.fit_transform(x)

class Predictor(Prep):
  def __init__(self, model, degree: int) -> None:
      self.model = model
      self.degree = degree

  def predict(self, x:list):
    x_ = self.prepx(x, self.degree)
    return self.model.predict(x_)

class Model:
  def __init__(self, x, y, degree, model):
    self.x = x
    self.y = y
    self.degree = degree
    self.model = model
    self.score = self.model.score(self.x, y)

class BestModel(Prep):
  def __init__(self, x:list, y:list) -> None:
      self.x = x
      self.y = np.array(y)
  
  def get_model(self, degree:int):
    x_ = self.prepx(self.x, degree)
    model = LinearRegression().fit(x_, self.y)
    return Model(x_, self.y, degree, model)
  
  def compute_best_model(self):
    models = [self.get_model(i) for i in range(2,10)]
    model_scores = {m.degree: m.score for m in models}
    print(model_scores)
    best = None
    for m in models:
      if best is None:
        best = m
        continue
      if m.score == best.score:
        continue
      elif m.score == 1:
        break
      elif m.score >= 0.95 and best.score >= 0.95:
        tmp = {m: m.score, best: best.score}
        best = min(tmp, key=tmp.get)
      else:
        tmp = {m: m.score, best: best.score}
        best = max(tmp, key=tmp.get)

    print("best score =", best.score, "degree =", best.degree)
    return Predictor(best.model, best.degree)

In [47]:
x = [5, 15, 25, 35, 45, 55]
y = [5, 20, 14, 32, 22, 38]
my_obj = BestModel(x, y)
p = my_obj.compute_best_model()
p.predict([25])

{2: 0.7178963893249607, 3: 0.7665364017947607, 4: 0.770594653573377, 5: 0.9999999999999999, 6: 0.9999999999999264, 7: 1.0, 8: 1.0, 9: 1.0}
best score = 0.9999999999999264 degree = 6


array([13.99999984])

In [46]:
x = [
  [0, 1], [5, 1], [15, 2], [25, 5], [35, 11], [45, 15], [55, 34], [60, 35]
]
y = [4, 5, 20, 14, 32, 22, 38, 43]
my_obj = BestModel(x, y)
p = my_obj.compute_best_model()
p.predict([[25, 5]])

{2: 0.9453701449127822, 3: 1.0, 4: 1.0, 5: 1.0, 6: 1.0, 7: 1.0, 8: 0.9999999999999999, 9: 0.9999999999999843}
best score = 0.9453701449127822 degree = 2


array([15.79139])

2